# Importing Spark

In [1]:
import pyspark

from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Python Spark").getOrCreate()

# Loading the data

In [2]:
df_ratings = spark.read\
    .option("delimiter", "\t")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('data/u.data')

In [3]:
# print the dataframe schema
df_ratings.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [4]:
# show a sample of the data (the dataframe executes the whole pipeline at this stage)
df_ratings.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [5]:
# set the rdd equivalent of the dataframe
rdd_ratings = df_ratings.rdd

# Basic Queries

#### Exercice 1 - Number of movies per user (using RDD then Dataframe)

Calculer pour chaque utilisateur le nombre de films notés, et afficher le résultat pour l'un d'entre eux. Utilisez dans un premier temps les RDD puis les Dataframes.

In [6]:
# classical RDD approach
result_1 = rdd_ratings.map(lambda r: (r[0], 1)).reduceByKey(lambda v1, v2: v1 + v2).take(1)
print(result_1)

[(196, 39)]


In [7]:
# dataframe approach, filtering on the previous user to compare results
df_ratings.filter(df_ratings['user_id']==result_1[0][0])\
    .groupBy('user_id')\
    .count()\
    .show(1)

+-------+-----+
|user_id|count|
+-------+-----+
|    196|   39|
+-------+-----+



#### Exercice 2 - Average rating per user (using RDD then Dataframe)

Calculer pour chaque utilisateurs la note moyenne donnée et afficher le résultat pour l'un d'entre eux. Utilisez dans un premier temps les RDD puis les Dataframes.

In [8]:
# classical RDD approach
rdd_map = rdd_ratings.map(lambda r: (r[0], int(r[2])))
rdd_agg = rdd_map = rdd_map.aggregateByKey(
    (0, 0), 
    lambda a,b: (a[0] + b,    a[1] + 1), 
    lambda a,b: (a[0] + b[0], a[1] + b[1])
)

rdd_result = rdd_agg.mapValues(lambda v: float(v[0])/v[1])

result_1 = rdd_result.take(1)
print(result_1)

[(196, 3.6153846153846154)]


In [9]:
# dataframe approach
from pyspark.sql.functions import avg

df_ratings.filter(df_ratings['user_id']==result_1[0][0])\
    .groupBy('user_id')\
    .agg(avg('rating'))\
    .show(1)

+-------+------------------+
|user_id|       avg(rating)|
+-------+------------------+
|    196|3.6153846153846154|
+-------+------------------+



#### Exercice 3 - Top-5 movies with at least 15 votes (Dataframe)

Afficher les 5 meilleurs films parmi ceux qui ont reçu au moins 15 votes.

*Indices:*
* Utiliser df_ratings pour calculer la moyenne, filtrer les films qui ont moins de 15 notes et classer les films par ordre décroissant.
* Faire un join avec df_movies pour afficher le nom des films sélectionnés.

In [10]:
df_items = spark.read\
    .option("delimiter", "|")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('data/u.item')

In [11]:
from pyspark.sql.functions import avg, count, col

df_gb = df_ratings.groupBy('item_id')\
    .agg(avg('rating'), count('item_id').alias('count'))

In [15]:
df_gb = df_gb.filter(df_gb['count'] >= 15).sort("avg(rating)", ascending=False)

In [16]:
# join with actual movie features
df_join = df_gb.join(df_items, df_gb['item_id']==df_items['movie_id'])

In [17]:
df_join.select(col("movie_title"), col("avg(rating)")).show(5)  # Java 8 (does not work with Java 12)

+--------------------+-----------------+
|         movie_title|      avg(rating)|
+--------------------+-----------------+
|Close Shave, A (1...|4.491071428571429|
|Schindler's List ...|4.466442953020135|
|Wrong Trousers, T...|4.466101694915254|
|   Casablanca (1942)| 4.45679012345679|
|Wallace & Gromit:...|4.447761194029851|
+--------------------+-----------------+
only showing top 5 rows

